In [ ]:
# Import necessary libraries
# import numpy as np
# import os
# import pandas as pd
# from sklearn import preprocessing
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt

In [ ]:
# Load the Olivetti Faces dataset
data = fetch_olivetti_faces(shuffle=True, random_state=42)
faces = data.data
labels = data.target

In [ ]:
# Print information about the dataset
print(f"Number of samples: {faces.shape[0]}")
print(f"Number of features (pixels): {faces.shape[1]}")
print(f"Number of unique labels (people): {len(set(labels))}")

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(faces, labels, test_size=0.2, random_state=42)


# Use PCA for dimensionality reduction
n_components = 150
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
# Train a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier()
decision_tree_classifier.fit(X_train_pca, y_train)

# Predict using the trained Decision Tree model
y_pred_tree = decision_tree_classifier.predict(X_test_pca)

# Evaluate the Decision Tree model
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print(f"Decision Tree Accuracy: {accuracy_tree:.2f}")

In [ ]:
# Train a Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_classifier.fit(X_train_pca, y_train)

# Predict using the trained Random Forest model
y_pred_forest = random_forest_classifier.predict(X_test_pca)

# Evaluate the Random Forest model
accuracy_forest = accuracy_score(y_test, y_pred_forest)
print(f"Random Forest Accuracy: {accuracy_forest:.2f}")

In [ ]:
# Show Example:
plt.imshow(data.images[0], cmap="gray")
plt.show()

In [ ]:
# # Load an example image for testing
# example_image_path = ""
# example_image = cv2.imread(example_image_path, cv2.IMREAD_GRAYSCALE)

# # Resize the image to match the training images' size (64x64)
# example_image = cv2.resize(example_image, (64, 64))

# # Flatten the image to match the model's input shape
# example_image_flattened = example_image.flatten().reshape(1, -1)

# # Perform PCA on the example image
# example_image_pca = pca.transform(example_image_flattened)

# # Predict using the trained Random Forest model
# predicted_label = random_forest_classifier.predict(example_image_pca)[0]

# # Get the predicted person's name based on the label
# predicted_name = f"Person {predicted_label}"

# # Display the input image along with the predicted name
# plt.imshow(example_image, cmap='gray')
# plt.title(predicted_name)
# plt.axis('off')
# plt.show()